# Project 1

In [5]:
import numpy as np
import sklearn as sk
# import gensim
import nltk

## 1 - Dataset Preparation & Analysis

### 1.2 - Loading Dataset

In [10]:
import json

with open('goemotions.json') as f:
	dataset = json.load(f)
	
print(len(dataset))

171820
